In [1]:
from openai import OpenAI
import os
import json
## initialize openai client
client = OpenAI(api_key = os.getenv('OPEN_AI_API_KEY'))


In [14]:
def let_gpt_judge_name(name, sql, model):

    cte_eval_prompt = f"""
        You will evaluate the comprehensiveness of the name assigned to this common table expression (CTE) from 1 to 10, where:
        - 1 indicates a name that is vague or unrelated, and
        - 10 indicates a highly descriptive, accurate name that clearly reflects the CTE's purpose.

        Examples:

        1. SQL: SELECT customer_id, SUM(order_total) AS total_spent FROM orders GROUP BY customer_id
            - Name: total_spent_per_customer – Expected score: 9
            - Name: customer_data – Expected score: 3
        2. SQL: SELECT product_id, COUNT(*) AS order_count FROM order_details GROUP BY product_id
            - Name: ProductOrderCounts – Expected score: 10
            - Name: details – Expected score: 2
        3. SQL: SELECT order_id, MAX(order_date) FROM orders GROUP BY order_id
            - Name: latest_order_dates – Expected score: 8
            - Name: order_info – Expected score: 4

        Placeholder names like tmp, aaa, cte1, table1, mycte, etc. are placeholder names that say nothing about the CTE, so they would recieve scores of 1-2.

        Assume the following SQL snippet:
        ```sql
        {sql}
        ```
        Your task: Evaluate this proposed name for the SQL statement in terms of its comprehensiveness. Provide only a single numeric score from 1 to 10 as output.

        Name to evaluate: {name}

        """
    
    persona =  "graduate student in computer science specializing in databases and skilled in SQL" 

    messages = [
    {"role": "system",
     "content": f"You are a {persona}, and you're tasked with evaluating names to common table expressions. The names should reflect what result set the SQL is trying to produce"},
    {"role": "user",
     "content": cte_eval_prompt},
    ]

    model_response = client.chat.completions.create(model=model, messages=messages, max_tokens=50, seed=42, temperature=0) # "gpt-4o" or "gpt-4-turbo"
    
    return model_response.choices[0].message.content


In [10]:
def write_in_gpt_evaluations(json_cte_data_file):
    with open(json_cte_data_file, 'r+') as file:
        cte_data = json.load(file)

        for i, obj in enumerate(cte_data):
            if "gpt-4o-name-evaluation" not in obj or not obj["gpt-4o-name-evaluation" ]:
                try:
                    obj["gpt-4o-name-evaluation" ] = let_gpt_judge_name(obj["cte_name"], obj['SQL'], "gpt-4o")
                    file.seek(0)
                    json.dump(cte_data, file, indent=2)
                    file.truncate()
                except Exception as e:
                    print(f"Error generating 4o summary for {json_cte_data_file} at index {i}: {e}")
                    # Pause and return to allow rerun later
                    return -1
            if "gpt-4-turbo-name-evaluation" not in obj or not obj["gpt-4-turbo-name-evaluation" ]:
                try:
                    obj["gpt-4-turbo-name-evaluation" ] = let_gpt_judge_name(obj["cte_name"], obj['SQL'], "gpt-4-turbo")
                    file.seek(0)
                    json.dump(cte_data, file, indent=2)
                    file.truncate()
                except Exception as e:
                    print(f"Error generating 4-turbo summary for {json_cte_data_file} at index {i}: {e}")
                    # Pause and return to allow rerun later
                    return -1
    return 0

In [15]:
data_path = 'data_files/curated_ctesjson'

write_in_gpt_evaluations(data_path)

0